# Ashbard Energy — Production Efficiency Tracker

This notebook performs EDA, KPI calculations, anomaly detection, and short-term forecasting for the synthetic Ashbard Energy and Car Dealership datasets. It is structured so you can run each section sequentially in JupyterLab or Colab.

## 1. Setup & Packages
Install necessary packages (run if missing).

In [ ]:
# Optional installs (uncomment if needed)
# %pip install prophet scikit-learn matplotlib pandas streamlit nbconvert seaborn plotly

import pandas as pd, numpy as np
print('pandas', pd.__version__)
print('numpy', np.__version__)


## 2. Load data

In [ ]:
ashbard = pd.read_csv(r"/mnt/data/ashbard_production.csv", parse_dates=['date'])
dealership = pd.read_csv(r"/mnt/data/dealership_energy.csv", parse_dates=['date'])
print('Ashbard rows, cols:', ashbard.shape)
print('Dealership rows, cols:', dealership.shape)
display(ashbard.head())
display(dealership.head())


## 3. Basic Cleaning & Feature Engineering

In [ ]:
def prepare_ashbard(df):
    df = df.copy()
    df = df.sort_values('date')
    df['oil_bbl'] = pd.to_numeric(df['oil_bbl'], errors='coerce')
    df['gas_mcf'] = pd.to_numeric(df['gas_mcf'], errors='coerce')
    df['production_boe'] = pd.to_numeric(df['production_boe'], errors='coerce')
    df['energy_kwh'] = pd.to_numeric(df['energy_kwh'], errors='coerce')
    df['energy_per_boe'] = df['energy_kwh'] / (df['production_boe'].replace(0, pd.NA))
    df['operating_cost_per_boe'] = df['operating_cost_usd'] / (df['production_boe'].replace(0, pd.NA))
    df['month'] = df['date'].dt.to_period('M')
    df['dayofweek'] = df['date'].dt.day_name()
    return df

ash = prepare_ashbard(ashbard)
dl = dealership.copy()
dl['month'] = dl['date'].dt.to_period('M')
display(ash[['date','well_id','production_boe','energy_kwh','energy_per_boe']].head())


## 4. Exploratory Data Analysis (visuals)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10,4)

daily = ash.groupby('date').agg({'production_boe':'sum','energy_kwh':'sum'}).reset_index()
daily['rolling_prod_7d'] = daily['production_boe'].rolling(7, min_periods=1).mean()

fig, ax = plt.subplots()
ax.plot(daily['date'], daily['production_boe'], label='BOE/day')
ax.plot(daily['date'], daily['rolling_prod_7d'], label='7-day MA', linestyle='--')
ax.set_title('Total Production (BOE/day)')
ax.set_ylabel('BOE/day')
ax.legend()
plt.show()


## 5. KPI Calculations

In [ ]:
kpi_well = ash.groupby('well_id').agg(
    avg_boe = ('production_boe','mean'),
    total_boe = ('production_boe','sum'),
    avg_energy_kwh = ('energy_kwh','mean'),
    energy_per_boe = ('energy_per_boe','mean'),
    uptime_hours = ('uptime_hours','mean')
).reset_index().sort_values('total_boe', ascending=False)
display(kpi_well.head())

display(dl[['date','electricity_kwh','diesel_liters','fleet_miles','energy_cost_usd']].head())


## 6. Anomaly Detection

In [ ]:
from sklearn.ensemble import IsolationForest
anom_df = daily.copy().dropna()
X = anom_df[['production_boe','energy_kwh']].fillna(0)
iso = IsolationForest(contamination=0.03, random_state=42)
anom_df['anomaly_score'] = iso.fit_predict(X)
anom_df['is_anomaly'] = anom_df['anomaly_score'] == -1
print('Anomalies detected:', anom_df['is_anomaly'].sum())
display(anom_df[anom_df['is_anomaly']].head())


## 7. Forecasting (short-term)

In [ ]:
# Forecasting example using Prophet (install if missing)
# %pip install prophet
try:
    from prophet import Prophet
    use_prophet = True
except Exception as e:
    print('Prophet not available, will use statsmodels SARIMAX fallback. Error:', e)
    use_prophet = False

ts = daily[['date','production_boe']].rename(columns={'date':'ds','production_boe':'y'})
ts = ts.set_index('ds').resample('D').sum().reset_index()

if use_prophet:
    m = Prophet(yearly_seasonality=False,daily_seasonality=False, weekly_seasonality=True)
    m.fit(ts)
    future = m.make_future_dataframe(periods=60)
    forecast = m.predict(future)
    display(forecast[['ds','yhat','yhat_lower','yhat_upper']].tail())
else:
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    series = ts.set_index('ds')['y']
    model = SARIMAX(series, order=(1,1,1), seasonal_order=(0,0,0,0))
    res = model.fit(disp=False)
    pred = res.get_forecast(60)
    pred_df = pred.summary_frame()
    display(pred_df.tail())


## 8. Export results & next steps

In [ ]:
import os
os.makedirs('outputs', exist_ok=True)
kpi_well.to_csv('outputs/kpi_well.csv', index=False)
anom_df[anom_df['is_anomaly']].to_csv('outputs/anomalies.csv', index=False)
if 'forecast' in globals():
    forecast.to_csv('outputs/forecast.csv', index=False)
print('Saved outputs to outputs/ folder in the notebook working directory.')
